In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

# ----------------------
# 1. Data Preparation
# ----------------------
# Load and merge data
looks_df = pd.read_csv('looks.csv')  # Columns: look_id, category, product_id
products_df = pd.read_csv('products.csv')  # Columns: product_id, product_name
merged_df = pd.merge(looks_df, products_df, on='product_id', how='left')

# Group products by look
look_products = merged_df.groupby('look_id').agg({
    'product_name': list,
    'category': list,
    'product_id': list
}).reset_index()

# ----------------------
# 2. Generate Embeddings
# ----------------------
# Initialize sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate product embeddings
products_df['product_embedding'] = products_df['product_name'].apply(
    lambda x: embedding_model.encode(x)

# Create look embeddings by averaging product embeddings
def get_look_embedding(product_ids):
    product_embs = products_df[products_df['product_id'].isin(product_ids)]['product_embedding']
    return np.mean(np.vstack(product_embs.values), axis=0)

look_products['look_embedding'] = look_products['product_id'].apply(get_look_embedding)

# ----------------------
# 3. PyTorch Model
# ----------------------
class RecommendationModel(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=256, output_dim=128):
        super(RecommendationModel, self).__init__()
        self.query_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)

        self.look_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim))

        def forward(self, query, look):
            query_emb = self.query_encoder(query)
        look_emb = self.look_encoder(look)
        return torch.cosine_similarity(query_emb, look_emb, dim=1)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RecommendationModel().to(device)

# ----------------------
# 4. Training (Optional)
# ----------------------
# Convert embeddings to PyTorch tensors
look_embeddings = np.vstack(look_products['look_embedding'])
X_train = torch.FloatTensor(look_embeddings).to(device)

# Synthetic training example (replace with real data)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()

    # Dummy training: make look embeddings match themselves
    outputs = model(X_train, X_train)
    loss = criterion(outputs, torch.ones_like(outputs))  # Target = perfect similarity

    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# ----------------------
# 5. Recommendation System
# ----------------------
def get_recommendations(user_query, top_k=5, product_count=10):
    # Encode user query
    query_embedding = embedding_model.encode(user_query)
    query_tensor = torch.FloatTensor(query_embedding).unsqueeze(0).to(device)

    # Prepare look embeddings
    look_embeddings = torch.FloatTensor(np.vstack(look_products['look_embedding'])).to(device)

    # Get similarities
    with torch.no_grad():
        model.eval()
        similarities = model(query_tensor.repeat(len(look_embeddings), 1), look_embeddings)

    # Get top-k looks
    _, indices = torch.topk(similarities, top_k)
    recommended_looks = look_products.iloc[indices.cpu().numpy()]

    # Aggregate products
    all_products = [pid for sublist in recommended_looks['product_id'] for pid in sublist]
    product_counts = Counter(all_products)

    # Get top products
    top_products = [item[0] for item in product_counts.most_common(product_count)]
    return products_df[products_df['product_id'].isin(top_products)]['product_name'].tolist()

# ----------------------
# 6. Example Usage
# ----------------------
if __name__ == "__main__":
    # Example query
    user_query = "casual summer outfit"

    # Get recommendations
    recommendations = get_recommendations(user_query)

    # Print results
    print(f"\nRecommendations for '{user_query}':")
    for idx, product in enumerate(recommendations, 1):
        print(f"{idx}. {product}")